In [13]:
%pip install Tweepy

     |████████████████████████████████| 153kB 2.9MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
import tweepy

In [3]:
OAUTH_TOKEN='1204477616398503936-7Z5WSIa0490S7HbIoPrDzosdA8aJKb'
OAUTH_SECRET='OCHWxZGQlrnaoNx4corTWHNZPrtGtItteYnow7nMDhtON'
CONSUMER_KEY='IRuah0diiQDx7ayuHaFmCOvTq'
CONSUMER_SECRET='FkYZE9vZXWMgWvovRjcqQaixPleh6ADXdD3XU05ZsIkscSFzXd'

In [5]:
auth = tweepy.OAuthHandler(CONSUMER_KEY,CONSUMER_SECRET)
auth.set_access_token(OAUTH_TOKEN,OAUTH_SECRET)
api = tweepy.API(auth)

In [16]:
#Búsqueda de Tweets por palabra
def busqueda_tweets_palabra(palabra="Ryanair", lenguaje="es", numero_de_Tweets=200):
    return tweepy.Cursor(api.search, palabra, lang=lenguaje).items(numero_de_Tweets)

tweets=busqueda_tweets_palabra()
outtweets = [(tweet.created_at, 
              tweet.text, 
              tweet.retweet_count, 
              str(tweet.favorite_count)+'') for tweet in tweets]
outtweets

[(datetime.datetime(2019, 12, 18, 19, 3, 53),
  'Ryanair atribuye los recortes en Girona a los problemas del Boeing 737 MAX https://t.co/asx2LZmyV3 vía @elpais_economia',
  0,
  '0'),
 (datetime.datetime(2019, 12, 18, 19, 0, 8),
  'Los asientos separados, el nuevo mercadillo de las low cost (Ryanair, Vueling, etc...) https://t.co/Zg75Rhm4re',
  0,
  '0'),
 (datetime.datetime(2019, 12, 18, 18, 49, 18),
  'RT @marcosleyley: He visto sardinas en lata que tienen más espacio que volar en Vueling o Ryanair. Es curioso que mientras la población mun…',
  3,
  '0'),
 (datetime.datetime(2019, 12, 18, 18, 48, 17),
  'si tienes cualquier problema para que puedas resolverlo "perfectamente" te ponen un número de atención al cliente d… https://t.co/e94lP2nyZ5',
  0,
  '1'),
 (datetime.datetime(2019, 12, 18, 18, 40),
  'viajar con @Ryanair_ES y otras maneras de solo tener problemas.',
  0,
  '2'),
 (datetime.datetime(2019, 12, 18, 18, 36, 2),
  'Inspección de Trabajo abrirá expediente ✈️ https://t.co/